In [62]:
import configparser
import os
import sys

In [63]:
from pyspark.sql import SparkSession

In [64]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [65]:
def process_deputies_data(spark, input_data, output_data):
    deputies_data = input_data + "deputies_dataset.csv"
    df = spark.read.csv(deputies_data, header=True, inferSchema=True)
    df.show(n=15)
    
    deputies_table = df.select('deputy_id', 'deputy_name').distinct()
    deputies_table.show(n=5)
    
    deputies_table.write.mode('overwrite').parquet(output_data + 'deputies')


In [60]:
def check_results(spark, output_data):
    df = spark.read.parquet(output_data + 'deputies')
    df.select('deputy_id', 'deputy_name')

In [44]:
def main():
    if len(sys.argv) == 3:
        # aws cluster mode
        input_data = sys.argv[1]
        output_data = sys.argv[2]
    else:
        # local mode
        config = configparser.ConfigParser()
        config.read('./airflow_home/dl.cfg')

        os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
        os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

        input_data = config['DATALAKE']['INPUT_DATA']
        print(input_data)
        output_data = './output'#config['DATALAKE']['OUTPUT_DATA']
        spark = create_spark_session()

        process_deputies_data(spark, input_data, output_data)
        check_results(spark, output_data)

In [43]:
main()

s3a://brazilian-deputies/
+-----------+-------------------+---------+---------------+----------+------------------+------------------------------+--------------------+--------------------+-------------+
|bugged_date|       receipt_date|deputy_id|political_party|state_code|       deputy_name|receipt_social_security_number| receipt_description|  establishment_name|receipt_value|
+-----------+-------------------+---------+---------------+----------+------------------+------------------------------+--------------------+--------------------+-------------+
|          0|2013-03-27 00:00:00|     1772|            PSB|        SP|Abelardo Camarinha|             3.530749000139E12|Fuels and lubrica...|AUTO POSTO 314 NO...|           70|
|          0|2013-07-24 00:00:00|     1772|            PSB|        SP|Abelardo Camarinha|             8.202116000115E12|Fuels and lubrica...|AUTO POSTO AEROPO...|          104|
|          0|2013-02-17 00:00:00|     1772|            PSB|        SP|Abelardo Camarinha|

In [61]:
output_data = './output'#config['DATALAKE']['OUTPUT_DATA']
spark = create_spark_session()


AnalysisException: "cannot resolve '`ASsda`' given input columns: [deputy_id, deputy_name];;\n'Project ['ASsda, deputy_name#427]\n+- Relation[deputy_id#426,deputy_name#427] parquet\n"